<a href="https://colab.research.google.com/github/sujalsin/Northstar/blob/master/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pre Processing

Download Data from - https://www.kaggle.com/datasets/bryanpark/nana-dataset?resource=download

From NaNa dataset- Upload **train.src, train.tgt, test.src, test.tgt**

Also, upload **Homograph.txt, checkpoint20.npz**

In [ ]:
import csv
names = open("/content/train.src", "r")
nats = open("/content/train.tgt", "r")
with open("/content/train.csv", "w") as f:
  writer = csv.writer(f)
  writer.writerow(["Names", "Nationality"])
  for name, nat in zip(names, nats):
    writer.writerow([name.replace("\n", ""), nat.replace("\n", "")])

In [ ]:
names = open("/content/test.src", "r")
nats = open("/content/test.tgt", "r")
dic = defaultdict()
dic2 = defaultdict()
with open("/content/test.csv", "w") as f:
  writer = csv.writer(f)
  writer.writerow(["Names", "Nationality"])
  for name, nat in zip(names, nats):
    #print(name.replace("\n", "").split(' ')[-1], nat)
    writer.writerow([name.replace("\n", ""), nat.replace("\n", "")])
    dic[name.replace("\n", "").split(' ')[-1]] = nat.replace("\n", "")
    dic2[name.replace("\n", "")] = nat.replace("\n", "")

In [ ]:
def lst_of_country():
  dic = defaultdict()
  country_vector = defaultdict()
  i = 0
  with open("/content/train.csv", 'r') as f:
    reader = csv.reader(f)
    header = next(reader)
    for line in reader:
      country = line[1].replace("\n", "")
      if country in dic:
        dic[country].append(line[0].split(" ")[-1])
      else:
        country_vector[country] = i
        dic[country] = [line[0].split(" ")[-1]]
        i += 1
  return country_vector

In [ ]:
from collections import defaultdict

country_vector = lst_of_country()
for key in country_vector.keys():
  print(key, country_vector[key])

NUMPY

In [ ]:
from __future__ import print_function
import inflect
import re



_inflect = inflect.engine()
_comma_number_re = re.compile(r'([0-9][0-9\,]+[0-9])')
_decimal_number_re = re.compile(r'([0-9]+\.[0-9]+)')
_pounds_re = re.compile(r'£([0-9\,]*[0-9]+)')
_dollars_re = re.compile(r'\$([0-9\.\,]*[0-9]+)')
_ordinal_re = re.compile(r'[0-9]+(st|nd|rd|th)')
_number_re = re.compile(r'[0-9]+')


def _remove_commas(m):
    return m.group(1).replace(',', '')


def _expand_decimal_point(m):
    return m.group(1).replace('.', ' point ')


def _expand_dollars(m):
    match = m.group(1)
    parts = match.split('.')
    if len(parts) > 2:
        return match + ' dollars'    # Unexpected format
    dollars = int(parts[0]) if parts[0] else 0
    cents = int(parts[1]) if len(parts) > 1 and parts[1] else 0
    if dollars and cents:
        dollar_unit = 'dollar' if dollars == 1 else 'dollars'
        cent_unit = 'cent' if cents == 1 else 'cents'
        return '%s %s, %s %s' % (dollars, dollar_unit, cents, cent_unit)
    elif dollars:
        dollar_unit = 'dollar' if dollars == 1 else 'dollars'
        return '%s %s' % (dollars, dollar_unit)
    elif cents:
        cent_unit = 'cent' if cents == 1 else 'cents'
        return '%s %s' % (cents, cent_unit)
    else:
        return 'zero dollars'


def _expand_ordinal(m):
    return _inflect.number_to_words(m.group(0))


def _expand_number(m):
    num = int(m.group(0))
    if num > 1000 and num < 3000:
        if num == 2000:
            return 'two thousand'
        elif num > 2000 and num < 2010:
            return 'two thousand ' + _inflect.number_to_words(num % 100)
        elif num % 100 == 0:
            return _inflect.number_to_words(num // 100) + ' hundred'
        else:
            return _inflect.number_to_words(num, andword='', zero='oh', group=2).replace(', ', ' ')
    else:
        return _inflect.number_to_words(num, andword='')


def normalize_numbers(text):
    text = re.sub(_comma_number_re, _remove_commas, text)
    text = re.sub(_pounds_re, r'\1 pounds', text)
    text = re.sub(_dollars_re, _expand_dollars, text)
    text = re.sub(_decimal_number_re, _expand_decimal_point, text)
    text = re.sub(_ordinal_re, _expand_ordinal, text)
    text = re.sub(_number_re, _expand_number, text)
    return text

**G2P**

In [ ]:
f = open("/homograph.txt")
homograph = dict()
for line in f.readlines()[10:]:
  if line.startswith("#"):continue
  l = line.split("|")
  print(l)
  homograph[l[0].lower()] = (l[1].split(), l[2].split(), l[3].replace("\\\n", ""))
print(len(homograph))

In [ ]:
import numpy as np
variables = np.load("/checkpoint20.npz")
print(variables['enc_emb'])

In [ ]:
from nltk import pos_tag
from nltk.corpus import cmudict
import nltk
from nltk.tokenize import TweetTokenizer
word_tokenize = TweetTokenizer().tokenize
import numpy as np
import codecs
import re
import os
import unicodedata
from builtins import str as unicode

try:
    nltk.data.find('taggers/averaged_perceptron_tagger.zip')
except LookupError:
    nltk.download('averaged_perceptron_tagger')
try:
    nltk.data.find('corpora/cmudict.zip')
except LookupError:
    nltk.download('cmudict')



def construct_homograph_dictionary():
    homograph2features = dict()
    f = open("/homograph.txt")
    for line in f.readlines()[10:]:
        #print(line)
        if line.startswith("#"): continue # comment
        headword, pron1, pron2, pos1 = line.split("|")[0], line.split("|")[1],line.split("|")[2],line.split("|")[3].replace("\\\n", "")
        homograph2features[headword.lower()] = (pron1.split(), pron2.split(), pos1)
    return homograph2features

# def segment(text):
#     '''
#     Splits text into `tokens`.
#     :param text: A string.
#     :return: A list of tokens (string).
#     '''
#     print(text)
#     text = re.sub('([.,?!]( |$))', r' \1', text)
#     print(text)
#     return text.split()

class G2p(object):
    def __init__(self):
        super().__init__()
        self.graphemes = ["<pad>", "<unk>", "</s>"] + list("abcdefghijklmnopqrstuvwxyz")
        self.phonemes = ["<pad>", "<unk>", "<s>", "</s>"] + ['AA0', 'AA1', 'AA2', 'AE0', 'AE1', 'AE2', 'AH0', 'AH1', 'AH2', 'AO0',
                                                             'AO1', 'AO2', 'AW0', 'AW1', 'AW2', 'AY0', 'AY1', 'AY2', 'B', 'CH', 'D', 'DH',
                                                             'EH0', 'EH1', 'EH2', 'ER0', 'ER1', 'ER2', 'EY0', 'EY1',
                                                             'EY2', 'F', 'G', 'HH',
                                                             'IH0', 'IH1', 'IH2', 'IY0', 'IY1', 'IY2', 'JH', 'K', 'L',
                                                             'M', 'N', 'NG', 'OW0', 'OW1',
                                                             'OW2', 'OY0', 'OY1', 'OY2', 'P', 'R', 'S', 'SH', 'T', 'TH',
                                                             'UH0', 'UH1', 'UH2', 'UW',
                                                             'UW0', 'UW1', 'UW2', 'V', 'W', 'Y', 'Z', 'ZH']
        self.g2idx = {g: idx for idx, g in enumerate(self.graphemes)}
        self.idx2g = {idx: g for idx, g in enumerate(self.graphemes)}

        self.p2idx = {p: idx for idx, p in enumerate(self.phonemes)}
        self.idx2p = {idx: p for idx, p in enumerate(self.phonemes)}

        self.cmu = cmudict.dict()
        self.load_variables()
        self.homograph2features = construct_homograph_dictionary()

    def load_variables(self):
        self.variables = np.load("/checkpoint20.npz")
        self.enc_emb = self.variables["enc_emb"]  # (29, 64). (len(graphemes), emb)
        self.enc_w_ih = self.variables["enc_w_ih"]  # (3*128, 64)
        self.enc_w_hh = self.variables["enc_w_hh"]  # (3*128, 128)
        self.enc_b_ih = self.variables["enc_b_ih"]  # (3*128,)
        self.enc_b_hh = self.variables["enc_b_hh"]  # (3*128,)

        self.dec_emb = self.variables["dec_emb"]  # (74, 64). (len(phonemes), emb)
        self.dec_w_ih = self.variables["dec_w_ih"]  # (3*128, 64)
        self.dec_w_hh = self.variables["dec_w_hh"]  # (3*128, 128)
        self.dec_b_ih = self.variables["dec_b_ih"]  # (3*128,)
        self.dec_b_hh = self.variables["dec_b_hh"]  # (3*128,)
        self.fc_w = self.variables["fc_w"]  # (74, 128)
        self.fc_b = self.variables["fc_b"]  # (74,)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x)) # mathematical logistic function // for the activation of the neural network

    def grucell(self, x, h, w_ih, w_hh, b_ih, b_hh):
        rzn_ih = np.matmul(x, w_ih.T) + b_ih # matrix_product of two array
        rzn_hh = np.matmul(h, w_hh.T) + b_hh

        rz_ih, n_ih = rzn_ih[:, :rzn_ih.shape[-1] * 2 // 3], rzn_ih[:, rzn_ih.shape[-1] * 2 // 3:]
        rz_hh, n_hh = rzn_hh[:, :rzn_hh.shape[-1] * 2 // 3], rzn_hh[:, rzn_hh.shape[-1] * 2 // 3:]

        rz = self.sigmoid(rz_ih + rz_hh)
        r, z = np.split(rz, 2, -1)

        n = np.tanh(n_ih + r * n_hh)
        h = (1 - z) * n + z * h

        return h

    def gru(self, x, steps, w_ih, w_hh, b_ih, b_hh, h0=None):
        if h0 is None:
            h0 = np.zeros((x.shape[0], w_hh.shape[1]), np.float32)
        h = h0  # initial hidden state
        outputs = np.zeros((x.shape[0], steps, w_hh.shape[1]), np.float32)
        for t in range(steps):
            h = self.grucell(x[:, t, :], h, w_ih, w_hh, b_ih, b_hh)  # (b, h)
            outputs[:, t, ::] = h
        return outputs

    def encode(self, word):
        chars = list(word) + ["</s>"]
        x = [self.g2idx.get(char, self.g2idx["<unk>"]) for char in chars]
        x = np.take(self.enc_emb, np.expand_dims(x, 0), axis=0)

        return x

    def predict(self, word):
        # encoder
        enc = self.encode(word)
        enc = self.gru(enc, len(word) + 1, self.enc_w_ih, self.enc_w_hh,
                       self.enc_b_ih, self.enc_b_hh, h0=np.zeros((1, self.enc_w_hh.shape[-1]), np.float32))
        last_hidden = enc[:, -1, :]

        # decoder
        dec = np.take(self.dec_emb, [2], axis=0)  # 2: <s>
        h = last_hidden

        preds = []
        for i in range(20):
            h = self.grucell(dec, h, self.dec_w_ih, self.dec_w_hh, self.dec_b_ih, self.dec_b_hh)  # (b, h)
            logits = np.matmul(h, self.fc_w.T) + self.fc_b
            pred = logits.argmax()
            if pred == 3: break  # 3: </s>
            preds.append(pred)
            dec = np.take(self.dec_emb, [pred], axis=0)

        preds = [self.idx2p.get(idx, "<unk>") for idx in preds]
        return preds

    def __call__(self, text):
        # preprocessing
        text = unicode(text)
        text = normalize_numbers(text)
        text = ''.join(char for char in unicodedata.normalize('NFD', text) #Convert the text into its decomposed form
                       if unicodedata.category(char) != 'Mn')  # Strip accents
        text = text.lower()
        text = re.sub("[^ a-z'.,?!\-]", "", text)
        text = text.replace("i.e.", "that is")
        text = text.replace("e.g.", "for example")

        # tokenization
        words = word_tokenize(text)
        tokens = pos_tag(words)  # tuples of (word, tag)

        # steps
        prons = []
        for word, pos in tokens:
            if re.search("[a-z]", word) is None:
                pron = [word]

            elif word in self.homograph2features:  # Check homograph
                pron1, pron2, pos1 = self.homograph2features[word]
                if pos.startswith(pos1):
                    pron = pron1
                else:
                    pron = pron2
            elif word in self.cmu:  # lookup CMU dict
                pron = self.cmu[word][0]
            else: # predict for oov
                pron = self.predict(word)

            prons.extend(pron)
            prons.extend([" "])

        return prons[:-1]

if __name__ == '__main__':
    texts = ["I have $250 in my pocket.", # number -> spell-out
             "popular pets, e.g. cats and dogs", # e.g. -> for example
             "I refuse to collect the refuse around here.", # homograph
             "I'm an activationist."] # newly coined word
    g2p = G2p()
    for text in texts:
        out = g2p(text)
        print(out)
    text = "Robert Pruzan"
    print(g2p(text))

**Name2Nationality**

In [ ]:
from traitlets.traitlets import default
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from collections import defaultdict
import csv
name_to_country2 = defaultdict()
name_to_index2 = defaultdict()
g2p = G2p()
def get_names():
  lst = []
  with open("/content/train.csv", 'r') as f:
    reader = csv.reader(f)
    header = next(reader)
    i = 0
    for line in reader:
      name_to_country2["".join(g2p(line[0]))] = line[1]
      name_to_index2["".join(g2p(line[0]))] = i
      i += 1
      lst.append("".join(g2p(line[0])))
      if i == 1000:
        break
  return lst

names = get_names()

In [ ]:
def train_model(epochs, vector_size, window):
    documents = [TaggedDocument(list(doc), [i]) for i, doc in enumerate(names)]
    model = Doc2Vec(documents, epochs=epochs, vector_size=vector_size, window=window, workers=1)
    return model
model2 = train_model(50, 50, 4)

name_to_vector2 = defaultdict()

for i in name_to_index2.keys():
  name_to_vector2[i] = model2.docvecs[name_to_index2[i]]

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
lst2 = []
y2 = []
country_vector = lst_of_country()
for i in name_to_index2.keys():
  name_to_vector2[i] = model2.docvecs[name_to_index2[i]]
  y2.append(country_vector[name_to_country2[i]])
  lst2.append(name_to_vector2[i].flatten())
neigh2 = KNeighborsClassifier(n_neighbors=30)
neigh2.fit(lst2, y2)
n = "".join(g2p('Li you'))
name_vector2 = model2.infer_vector(n).flatten()
print(neigh2.predict([name_vector2]))
for i in country_vector.keys():
  print(i, country_vector[i])

To Check Accuracy

In [ ]:
i = 0
j = 0
for key in list(dic2.keys())[:2000]:

  phoneme = g2p(key)
  name_vector = model2.infer_vector("".join(phoneme)).flatten()
  vector = neigh2.predict([name_vector])
  if vector[0] == country_vector[dic2[key]]:
    i += 1
  j += 1

print(float(i/j))